In [81]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data/', one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


- learning_rate = 최적화 함수에서 사용할 학습률
- training_epoch = 전체 데이터를 학습할 총횟수
- batch_size = 미니배치로 한번에 학습할 데이터의 개수
- n_hidden = 은닉층의 뉴런 개수
- n_input = 입력값의 크기


In [82]:
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28 * 28
n_noise = 128

In [83]:
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

In [84]:
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

In [85]:
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

In [86]:
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    
    return output

In [87]:
def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    
    return output

In [88]:
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [89]:
G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)

In [90]:
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))

In [91]:
loss_G = tf.reduce_mean(tf.log(D_gene))

In [92]:
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

In [93]:
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)

In [96]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        _, loss_val_D = sess.run([train_D, loss_D],
                              feed_dict={X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G],
                              feed_dict={Z: noise})
        
    print('Epoch:', '%4d' % epoch,
         'D loss:', '{:.4}'.format(loss_val_D),
         'G loss:', '{:.4}'.format(loss_val_G))
    
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})
        
        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))
        
        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))
            
        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)),
                               bbox_inches='tight')
        plt.close(fig)
        
print('finish!')

Epoch:    0 D loss: -0.4606 G loss: -2.149
Epoch:    1 D loss: -0.2708 G loss: -2.535
Epoch:    2 D loss: -0.09213 G loss: -3.532
Epoch:    3 D loss: -0.4732 G loss: -1.537
Epoch:    4 D loss: -0.3575 G loss: -1.86
Epoch:    5 D loss: -0.4892 G loss: -1.919
Epoch:    6 D loss: -0.3289 G loss: -2.435
Epoch:    7 D loss: -0.4689 G loss: -2.318
Epoch:    8 D loss: -0.302 G loss: -2.575
Epoch:    9 D loss: -0.4112 G loss: -2.195
Epoch:   10 D loss: -0.321 G loss: -2.703
Epoch:   11 D loss: -0.4394 G loss: -2.368
Epoch:   12 D loss: -0.3339 G loss: -2.425
Epoch:   13 D loss: -0.516 G loss: -2.034
Epoch:   14 D loss: -0.4579 G loss: -2.446
Epoch:   15 D loss: -0.6029 G loss: -1.922
Epoch:   16 D loss: -0.4698 G loss: -2.162
Epoch:   17 D loss: -0.3778 G loss: -2.167
Epoch:   18 D loss: -0.4896 G loss: -2.136
Epoch:   19 D loss: -0.5377 G loss: -1.899
Epoch:   20 D loss: -0.5769 G loss: -1.849
Epoch:   21 D loss: -0.5748 G loss: -1.931
Epoch:   22 D loss: -0.5777 G loss: -2.312
Epoch:   23 D 